# The Ray Tracer Challenge
## A Test-Driven Guide to Your First 3D Renderer
### by James Buck

>Brace yourself for a fun challenge: build a photorealistic 3D renderer from scratch! It’s easier than you think. In just a couple of weeks, build a ray tracer that renders beautiful scenes with shadows, reflections, brilliant refraction effects, and subjects composed of various graphics primitives: spheres, cubes, cylinders, triangles, and more. With each chapter, implement another piece of the puzzle and move the renderer that much further forward. Do all of this in whichever language and environment you prefer, and do it entirely test-first, so you know it’s correct. Recharge yourself with this project’s immense potential for personal exploration, experimentation, and discovery.

Opening quote from the beta ebook copy, September 2018. Publication due November 2018 (now February 2019). For details, see

https://pragprog.com/book/jbtracer/the-ray-tracer-challenge

This Jupyter notebook will showcase the demos at the end of each book chapter. The PPM graphic files can be viewed by the GIMP graphic program. GIMP was used to export each ray tracer generated file as a GIF file for display in this notebook.

# Workspace Management
I can load and save a workspace, which is the means by which I can separate notebooks containing test cases from a code development notebook. One thing seems notable; reducing the number of cells in a notebook reduces the time it takes to load the notebook. I've been combining multiple related test cases into a single markdown cell and their implementation in another code cell. Well worth continuing to do so.

In [1]:
⊃1 ⎕NPARTS ''
)WSID rtcode_ws
)load rtcode_ws
)CS
)objects
)fns
)vars
⎕PATH←'#.RayTracer'

C:/Users/Mark/Jupyter/

was

.\rtcode_ws.dws saved Fri Mar 22 17:53:06 2019

#

RayTracer

Some possible utility functions.

In [2]:
 )copy dfns time

E:\Mark\APL\Dyalog\APL_Files\dfns.dws saved Sat May 5 02:58:11 2018

# Changes and Updates
I'm using Reach Indexing now to alter material settings for objects. Previously I would create a custom material and assign it to an object. What's more, I would index attributes one at a time in some of the earlier demos and test cases. Not too efficient!

# Chapter 3 Demo
This demo is a simple projectile simulation plotted on a canvas.

In [3]:
)CS #
⎕PATH←'RayTracer'
projectile←{(⊂⍺),⊂⍵}
wrld←{(⊂⍺),⊂⍵}

#

In [4]:
∇ Z← W Tick P;position;velocity
position←(⊃P[1])+⊃P[2]
velocity←(⊃P[2])+(⊃W[1])+⊃W[2]
Z← position projectile velocity
∇

In [5]:
start← point 0 1 0
velocity← 11.25×normalize vector 1 1.8 0
+p← start projectile velocity
gravity← vector 0 ¯0.1 0
wind← vector ¯0.01 0 0
+w← gravity wrld wind

┌───────┬───────────────────────────┐
│0 1 0 1│5.463482976 9.834269356 0 0│
└───────┴───────────────────────────┘

┌──────────┬───────────┐
│0 ¯0.1 0 0│¯0.01 0 0 0│
└──────────┴───────────┘

In [6]:
∇Z← trajectory Params ;W;P;X;Y;c;limit;x;y
  (W P X Y)← Params
  c← canvas X Y
  limit← 1
  height← Y
  :While (limit<300)^0<0 1 0 0/⊃1↑P
    P← W Tick P
    (x y)← ⌊0.5+1 1 0 0/⊃1↑P
    x← 1⌈X⌊x ⋄ y← 1⌈Y⌊height-y
    ⍝ c[y;x]← ⊂1 0 0
    ((y x)⊃c)← 1 0 0
    limit +← 1
  :Endwhile
  Z← canvas_to_ppm c
∇

In [7]:
ppm← trajectory w p 900 550
ppm[1 2 3;]
'trajectory.ppm' savePPM ppm

┌───────┐
│P3 │
├───────┤
│900 550│
├───────┤
│255 │
└───────┘

6237189

And the resulting image is, after conversion from ppm to gif by GIMP:
<img src="trajectory.gif">

# Chapter 4 Demo
This demo uses rotation to compute the positions of the hours on a clock face.

In [8]:
∇ Z←clock C;r;c;cl;cx;cy;p2;p;idx;x;y;t
   (r c)← ⍴C
   cl← ¯10+⌊0.5×r⌈c
   (cx cy)← ⌊0.5×⍴C
   p2←p← point 0 cl 0
   :For idx :In ⍳12
      x← ⌊0.5+p2[1] ⋄ y← ⌊0.5+p2[2]
      ⍝ C[cx+x+¯2+⍳3;cy+y+¯2+⍳3]← ⊂1 1 1
      :For xx :In cx+x+¯2+⍳3
         :For yy :In cy+y+¯2+⍳3
            ((yy xx)⊃C)← 1 1 1
         :EndFor
      :EndFor
      t← rotation_z idx×○÷6
      p2← t +.× p
      :EndFor
   Z← canvas_to_ppm C
∇

In [9]:
)CS #
⎕PATH←'RayTracer'
c← canvas 200 200
ppm← clock c
'clock.ppm' savePPM ppm

#

504189

And here is the resulting image, converted to a GIF by GIMP
<img src="clock.gif">

# Chapter 5 Demo
Here's our first real ray tracing! This will not work as written because the structure of a canvas has changed. Update this later.

In [10]:
)CS #
⎕PATH←'RayTracer'
rayorigin← point 0 0 ¯5
wall_z← 10
wall_size← 7.0
canvas_pixels← 100
pixel_size← wall_size÷canvas_pixels
half← wall_size÷2
color← 1 0 0
ball← sphere

#

In [11]:
∇ Z← raycast C;row;col;pos;xs
  :For row :In ⍳canvas_pixels
    wy← half-pixel_size×row
    :For col :In ⍳canvas_pixels
      wx← (-half)+pixel_size×col
      pos← point wx wy wall_z
      r← rayorigin RayTracer.ray normalize pos-rayorigin
      xs← ball intersect r
      :If ⍬≢ hit xs
        ⍝ C[row;col]← color
        ((row col)⊃C)← color
        :EndIf
      :EndFor
    :EndFor
  Z← canvas_to_ppm C
∇

In [12]:
c← canvas canvas_pixels canvas_pixels
ppm← raycast c
'raytest.ppm' savePPM ppm

126189

And the results are (in gif format thanks to GIMP):
<img src="raytest.gif">

# Chapter 6 Demo
Take the code from the previous chapter that produced a silhouette of a sphere and show the actual sphere.

This demo will also not work as written until the code is altered to match the new canvas structure.

In [13]:
)CS #
⎕PATH←'RayTracer'
rayorigin← point 0 0 ¯5
wall_z← 10
wall_size← 7.0
canvas_pixels← 100
pixel_size← wall_size÷canvas_pixels
half← wall_size÷2
color← ⊂1 0 0
ball← sphere
⍝ Newly added settings
m← material
m[RayTracer.material_color]← ⊂1 0.2 1
ball[RayTracer.obj_material]← ⊂m
ball
+light← (point ¯10 10 ¯10) RayTracer.point_light 1 1 1

#

┌─┬───────┬───────┬───────────────────────────────┐
│1│1 0 0 0│1 0 0 0│┌───────┬───┬───┬───┬───┬─┬─┬─┐│
│ │0 1 0 0│0 1 0 0││1 0.2 1│0.1│0.9│0.9│200│0│1│0││
│ │0 0 1 0│0 0 1 0│└───────┴───┴───┴───┴───┴─┴─┴─┘│
│ │0 0 0 1│0 0 0 1│ │
└─┴───────┴───────┴───────────────────────────────┘

┌────────────┬─────┐
│¯10 10 ¯10 1│1 1 1│
└────────────┴─────┘

In [14]:
∇ Z← lightraycast C;row;col;world_y;world_x;pos;r;xs;rhit;s;t;pt;norm;mt
  :For row :In ⍳canvas_pixels
    world_y← half-pixel_size×row
    :For col :In ⍳canvas_pixels
      world_x← (-half)+pixel_size×col
      pos← point world_x world_y wall_z
      r← rayorigin RayTracer.ray normalize pos-rayorigin
      xs← ball intersect r
      :If 1<⍴ hit xs
        rhit← hit xs
        s←⊃rhit[RayTracer.hit_object]   ⍝ This mixed vector (sphere object) needs to be disclosed
        t← rhit[RayTracer.hit_distance]    ⍝ This scalar (distance) does not
        pt← r position t
        norm← s normal_at pt
        eye← -⊃r[RayTracer.ray_direction]  ⍝ This mixed array (vector) needs to be disclosed
        mt←⊃s[RayTracer.obj_material]     ⍝ This mixed array (material) needs to be disclosed
        color← lighting mt light pt eye norm 0 ball
        ⍝ C[row;col]← ⊂color
        ((row col)⊃C)← color
        :EndIf
      :EndFor
    :EndFor
  Z← canvas_to_ppm C
∇

In [15]:
c← canvas canvas_pixels canvas_pixels
ppm← lightraycast c
'lighttest.ppm' savePPM ppm

126189

And, the ray trace result looks like this

<img src="lighttest.gif">

# Chapter 7 Demo
This is a scene constructed entirely from Sphere objects. The walls are really just flattened spheres caused by the **scaling** function.

**Note:** I had to break the transform matrix assignment down into two steps to reduce the length of the line. A long line was causing a failure of the Jupyter APL kernel. There must be a line length limitation in the bridge between a notebook and the background APL interpreter.

In [16]:
)CS #
⎕PATH←'RayTracer'
floor← sphere
⍝ floor[obj_transform]← ⊂ scaling 10 0.01 10
floor← floor add_shape_transform scaling 10 0.01 10
m← material
m[RayTracer.material_color]← ⊂1 0.9 0.9
m[RayTracer.material_specular]← 0
floor[RayTracer.obj_material]← ⊂m

left_wall← sphere
⍝ left_wall[obj_transform]← ⊂(translation 0 0 5) +.× (rotation_y ○ ¯0.25) +.× (rotation_x ○ 0.5) +.× scaling 10 0.01 10
t← (translation 0 0 5) +.× (rotation_y ○ ¯0.25) +.× (rotation_x ○ 0.5) +.× scaling 10 0.01 10
left_wall← left_wall add_shape_transform t
left_wall[RayTracer.obj_material]← floor[RayTracer.obj_material]

right_wall← sphere
⍝ right_wall[obj_transform]← ⊂(translation 0 0 5) +.× (rotation_y ○ 0.25) +.× (rotation_x ○ 0.5) +.× scaling 10 0.01 10
t← (translation 0 0 5) +.× (rotation_y ○ 0.25) +.× (rotation_x ○ 0.5) +.× scaling 10 0.01 10
right_wall← right_wall add_shape_transform t
right_wall[RayTracer.obj_material]← floor[RayTracer.obj_material]

middle← sphere
⍝ middle[obj_transform]← ⊂translation ¯0.5 1 0.5
middle← middle add_shape_transform translation ¯0.5 1 0.5
m← material
m[RayTracer.material_color]← ⊂0.1 1 0.5
m[RayTracer.material_diffuse]← 0.7
m[RayTracer.material_specular]← 0.3
middle[RayTracer.obj_material]← ⊂m

right← sphere
⍝ right[obj_transform]← ⊂ (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
right← right add_shape_transform (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
m← material
m[RayTracer.material_color]← ⊂0.5 1 0.1
m[RayTracer.material_diffuse]← 0.7
m[RayTracer.material_specular]← 0.3
right[RayTracer.obj_material]← ⊂m

left← sphere
⍝ left[obj_transform]← ⊂ (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
left← left add_shape_transform (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
m← material
m[RayTracer.material_color]← ⊂1 0.8 0.1
m[RayTracer.material_diffuse]← 0.7
m[RayTracer.material_specular]← 0.3
left[RayTracer.obj_material]← ⊂m

w← world
w[1]← ⊂⊂ (point ¯10 10 ¯10) RayTracer.point_light 1 1 1
w[2]← ⊂ floor left_wall right_wall middle left right



#

In [17]:

c← camera 400 200 (○ ÷3)
c[RayTracer.camera_transform]← ⊂ view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)


In [18]:
im← w render #.time c

17.82

In [19]:
ppm← canvas_to_ppm im
'Chap7.ppm' savePPM ppm

1008189

Hurrah! Here's a rendered image, converted to GIF by GIMP.

<img src="Chap7.gif">

# Chapter 8 Demo
The demo is shown but not described. Here is a description from the official web site:<br>
http://forum.raytracerchallenge.com/thread/2/shadow-puppets-scene-description

```
# ======================================================
# puppets.yml
#
# This file describes the scene illustrated at the end
# of chapter 8, "Shadows", in "The Ray Tracer Challenge"
#
# by Jamis Buck <jamis@jamisbuck.org>
# ======================================================

# ======================================================
# the camera
# ======================================================

- add: camera
  width: 400
  height: 200
  field-of-view: 0.524
  from: [ 40, 0, -70 ]
  to: [ 0, 0, -5 ]
  up: [ 0, 1, 0 ]

# ======================================================
# light sources
# ======================================================

- add: light
  at: [ 0, 0, -100 ]
  intensity: [ 1, 1, 1 ]

# ======================================================
# define some constants to avoid duplication
# ======================================================

- define: sphere-material
  value:
    ambient: 0.2
    diffuse: 0.8
    specular: 0.3
    shininess: 200

- define: wrist-material
  extend: sphere-material
  value:
    color: [ 0.1, 1, 1 ]

- define: palm-material
  extend: sphere-material
  value:
    color: [ 0.1, 0.1, 1 ]

- define: thumb-material
  extend: sphere-material
  value:
    color: [ 0.1, 0.1, 1 ]

- define: index-material
  extend: sphere-material
  value:
    color: [ 1, 1, 0.1 ]

- define: middle-material
  extend: sphere-material
  value:
    color: [ 0.1, 1, 0.5 ]

- define: ring-material
  extend: sphere-material
  value:
    color: [ 0.1, 1, 0.1 ]

- define: pinky-material
  extend: sphere-material
  value:
    color: [ 0.1, 0.5, 1 ]

# ======================================================
# a backdrop onto which to cast the shadow
# ======================================================

- add: sphere
  material:
    color: [ 1, 1, 1 ]
    ambient: 0
    diffuse: 0.5
    specular: 0
  transform:
    - [ scale, 200, 200, 0.01 ]
    - [ translate, 0, 0, 20 ]

# ======================================================
# describe the elements of the scene
# ======================================================

# the wrist
- add: sphere
  material: wrist-material
  transform:
    - [ scale, 3, 3, 3 ]
    - [ translate, -4, 0, -21 ]
    - [ rotate-z, 0.785398163397448 ] # pi/4

# the palm
- add: sphere
  material: palm-material
  transform:
    - [ scale, 4, 3, 3 ]
    - [ translate, 0, 0, -15 ]

# the thumb
- add: sphere
  material: thumb-material
  transform:
    - [ scale, 1, 3, 1 ]
    - [ translate, -2, 2, -16 ]

# the index finger
- add: sphere
  material: index-material
  transform:
    - [ scale, 3, 0.75, 0.75 ]
    - [ translate, 3, 2, -22 ]

# the middle finger
- add: sphere
  material: middle-material
  transform:
    - [ scale, 3, 0.75, 0.75 ]
    - [ translate, 4, 1, -19 ]

# the ring finger
- add: sphere
  material: ring-material
  transform:
    - [ scale, 3, 0.75, 0.75 ]
    - [ translate, 4, 0, -18 ]

# the pinky finger
- add: sphere
  material: pinky-material
  transform:
    - [ scale, 2.5, 0.6, 0.6 ]
    - [ translate, 1, 0, 0 ]
    - [ rotate-z, -0.314159265358979 ] # pi/10
    - [ translate, 3, -1.5, -20]
```

In [20]:
)CS #
⎕PATH←'RayTracer'

sphere_material← material
sphere_material[RayTracer.material_ambient RayTracer.material_diffuse]← 0.2 0.8
sphere_material[RayTracer.material_specular RayTracer.material_shininess]← 0.3 200
wrist_material← sphere_material
wrist_material[RayTracer.material_color]←  ⊂0.1 1 1
palm_material← sphere_material
palm_material[RayTracer.material_color]←   ⊂0.1 0.1 1
thumb_material← sphere_material
thumb_material[RayTracer.material_color]←  ⊂0.1 0.1 1
index_material← sphere_material
index_material[RayTracer.material_color]←  ⊂1 1 0.1
middle_material← sphere_material
middle_material[RayTracer.material_color]← ⊂0.1 1 0.5
ring_material← sphere_material
ring_material[RayTracer.material_color]←   ⊂0.1 1 0.1
pinky_material← sphere_material
pinky_material[RayTracer.material_color]←  ⊂0.1 0.5 1

⍝ Backdrop
s1← sphere
m← material
m[RayTracer.material_color RayTracer.material_ambient]← (1 1 1) 0
m[RayTracer.material_diffuse RayTracer.material_specular]← 0.5 0
s1[RayTracer.obj_material]← ⊂m
⍝ s1[obj_transform]← ⊂(translation 0 0 20) +.× scaling 200 200 0.01
s1← s1 add_shape_transform (translation 0 0 20) +.× scaling 200 200 0.01
⍝ Wrist
s2← sphere
s2[RayTracer.obj_material]←  ⊂wrist_material
⍝ s2[obj_transform]← ⊂(rotation_z ○ 0.25) +.× (translation ¯4 0 ¯21) +.× scaling 3 3 3
s2← s2 add_shape_transform (rotation_z ○ 0.25) +.× (translation ¯4 0 ¯21) +.× scaling 3 3 3
⍝ Palm
s3← sphere
s3[RayTracer.obj_material]←  ⊂palm_material
⍝ s3[obj_transform]← ⊂(translation 0 0 ¯15) +.× scaling 4 3 3
s3← s3 add_shape_transform (translation 0 0 ¯15) +.× scaling 4 3 3
⍝ Thumb
s4← sphere
s4[RayTracer.obj_material]←  ⊂thumb_material
⍝ s4[obj_transform]← ⊂(translation ¯2 2 ¯16) +.× scaling 1 3 1
s4← s4 add_shape_transform (translation ¯2 2 ¯16) +.× scaling 1 3 1
⍝ Index finger
s5← sphere
s5[RayTracer.obj_material]←  ⊂index_material
⍝ s5[obj_transform]← ⊂(translation 3 2 ¯22) +.× scaling 3 0.75 0.75
s5← s5 add_shape_transform (translation 3 2 ¯22) +.× scaling 3 0.75 0.75
⍝ Middle finger
s6← sphere
s6[RayTracer.obj_material]←  ⊂middle_material
⍝ s6[obj_transform]← ⊂(translation 4 1 ¯19) +.× scaling 3 0.75 0.75
s6← s6 add_shape_transform (translation 4 1 ¯19) +.× scaling 3 0.75 0.75
⍝ Ring finger
s7← sphere
s7[RayTracer.obj_material]←  ⊂ring_material
⍝ s7[obj_transform]← ⊂(translation 4 0 ¯18) +.× scaling 3 0.75 0.75
s7← s7 add_shape_transform (translation 4 0 ¯18) +.× scaling 3 0.75 0.75
⍝ Pinky
s8← sphere
s8[RayTracer.obj_material]←  ⊂pinky_material
⍝ s8[obj_transform]← ⊂(translation 3 ¯1.5 ¯20) +.× (rotation_z ○ 0.1) +.× (translation 1 0 0) +.× scaling 2.5 0.6 0.6
s8← s8 add_shape_transform (translation 3 ¯1.5 ¯20) +.× (rotation_z ○ 0.1) +.× (translation 1 0 0) +.× scaling 2.5 0.6 0.6

w← world
w[1]← ⊂⊂ (point 0 0 ¯100) RayTracer.point_light 1 1 1
w[2]← ⊂ s1 s2 s3 s4 s5 s6 s7 s8

c← camera 400 200 0.524
c[RayTracer.camera_transform]← ⊂ view_transform (point 40 0 ¯70) (point 0 0 ¯5) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point 40 0 ¯70) (point 0 0 ¯5) (vector 0 1 0)

#

In [21]:
im← w render #.time c

19.86

In [22]:
ppm← canvas_to_ppm im
'puppets.ppm' savePPM ppm

1008189

Something's not quite right with the pinky (or is it?).

<img src="puppets.gif">

# Chapter 9 Demo
Use the three spheres from the chapter 7 demo and place them on a plane. For variation, add walls as well. For example, a plane rotated pi/2 about the x axis and translated a few units in the positive z direction.

In [23]:
)CS #
⎕PATH←'RayTracer'

floor← plane
⍝ m← material
⍝ m[material_color]← ⊂1 0.9 0.9
⍝ m[material_specular]← 0
⍝ floor[obj_material]← ⊂m
floor[(RayTracer.obj_material RayTracer.material_color)(RayTracer.obj_material RayTracer.material_specular)]← (1 0.9 0.9) 0

middle← sphere
⍝ middle[obj_transform]← ⊂translation ¯0.5 1 0.5
middle← middle add_shape_transform translation ¯0.5 1 0.5
⍝ m← material
⍝ m[material_color]← ⊂0.1 1 0.5
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ middle[obj_material]← ⊂m
idx←  ⊂(RayTracer.obj_material RayTracer.material_color)
idx,← ⊂(RayTracer.obj_material RayTracer.material_diffuse)
idx,← ⊂(RayTracer.obj_material RayTracer.material_specular)
middle[idx]← (0.1 1 0.5) 0.7 0.3

right← sphere
⍝ right[obj_transform]← ⊂ (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
right← right add_shape_transform (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
⍝ m← material
⍝ m[material_color]← ⊂0.5 1 0.1
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ right[obj_material]← ⊂m
right[idx]← (0.5 1 0.1) 0.7 0.3

left← sphere
⍝ left[obj_transform]← ⊂ (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
left← left add_shape_transform (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
⍝ m← material
⍝ m[material_color]← ⊂1 0.8 0.1
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ left[obj_material]← ⊂m
left[idx]← (1 0.8 0.1) 0.7 0.3

w← world
w[1]← ⊂⊂ (point ¯10 10 ¯10) RayTracer.point_light 1 1 1
w[2]← ⊂  floor middle left right

c← camera 400 200 (○ ÷3)
c[RayTracer.camera_transform]← ⊂ view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)


#

In [24]:
im← w render #.time c

10.77

In [25]:
ppm← canvas_to_ppm im
'Chap9.ppm' savePPM ppm

1008189

And here is the gif version of the Chapter 9 demo using a plane. All is now fixed with the **intersect** and **normal_at** functions.

<img src="Chap9.gif">

# Chapter 10 Demo
Here's the previous scene but with patterns applied to the objects in the scene.

In [26]:
)CS #
⎕PATH←'RayTracer'

floor← plane
m← material
⍝ m[material_color]←  ⊂1 0.9 0.9
m[RayTracer.material_color]←  ⊂0 0 0 gradient_pattern 1 1 1 ⍝ ⊂1 0.9 0.9
m[RayTracer.material_specular]← 0
floor[RayTracer.obj_material]← ⊂m

middle← sphere
⍝ middle[obj_transform]← ⊂translation ¯0.5 1 0.5
middle← middle add_shape_transform translation ¯0.5 1 0.5
m← material
⍝ m[material_color]← ⊂0.1 1 0.5
p← 1 0 1 checker_pattern 0 1 0
p← p add_pattern_transform scaling 0.1 0.1 0.1
⍝ m[material_color]← ⊂p
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ middle[obj_material]← ⊂m
idx←  ⊂(RayTracer.obj_material RayTracer.material_color)
idx,← ⊂(RayTracer.obj_material RayTracer.material_diffuse)
idx,← ⊂(RayTracer.obj_material RayTracer.material_specular)
middle[idx]← (p) 0.7 0.3

right← sphere
right← right add_shape_transform (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
⍝ m← material
⍝ m[material_color]← ⊂0.5 1 0.1
p← 0 1 0 ring_pattern 0 0 0
p← p add_pattern_transform (rotation_x ○ 0.5) +.× scaling 0.1 0.1 0.1
⍝ m[material_color]← ⊂p
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ right[obj_material]← ⊂m
right[idx]← (p) 0.7 0.3

left← sphere
left← left add_shape_transform (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
⍝ m← material
⍝ m[material_color]← ⊂1 0.8 0.1
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ left[obj_material]← ⊂m
left[idx]← (1 0.8 0.1) 0.7 0.3

w← world
w[1]← ⊂⊂ (point ¯10 10 ¯10) RayTracer.point_light 1 1 1
w[2]← ⊂  floor middle left right

c← camera 400 200 (○ ÷3)
c[RayTracer.camera_transform]← ⊂ view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)


#

In [27]:
im← w render #.time c

11.85

In [28]:
ppm← canvas_to_ppm im
'Chap10.ppm' savePPM ppm

1008189

The previous scene with patterns applied:

<img src="Chap10.gif">

# Chapter 11 Demo
Here's the scene from the book of two spheres above a checkered floor.

In [29]:
⍝ This version assumes ray tracer code is in the RayTracer namespace
)CS #
⎕PATH←'RayTracer'
floor← plane
p← 0 0 0 checker_pattern 1 1 1
⍝ p[RayTracer.pattern_transform]← ⊂translation 0 0.1 0
⍝ p[RayTracer.pattern_inverse]← ⊂⌹translation 0 0.1 0
p← p add_pattern_transform translation 0 0.1 0
m← material
m[RayTracer.material_pattern]← ⊂p
floor[RayTracer.obj_material]← ⊂m
⍝ floor[RayTracer.obj_transform]← ⊂translation 0 ¯4.1 0
⍝ floor[RayTracer.obj_inverse]← ⊂⌹translation 0 ¯4.1 0
floor← floor add_shape_transform translation 0 ¯4.1 0

s1← sphere
m← material
m[RayTracer.material_diffuse RayTracer.material_shininess RayTracer.material_reflective]← 0.1 300 1.0
m[RayTracer.material_transparency RayTracer.material_refractive]← 1 1.52
s1[RayTracer.obj_material]← ⊂m

s2← sphere
⍝ s2[RayTracer.obj_transform]← ⊂scaling 0.5 0.5 0.5
⍝ s2[RayTracer.obj_inverse]← ⊂⌹scaling 0.5 0.5 0.5
s2← s2 add_shape_transform scaling 0.5 0.5 0.5
m← material
m[RayTracer.material_diffuse RayTracer.material_shininess RayTracer.material_reflective]← 0.1 300 1.0
m[RayTracer.material_transparency RayTracer.material_refractive]← 1 1
s2[RayTracer.obj_material]← ⊂m

w← world
w[1]← ⊂⊂ (point ¯10 10 ¯10) RayTracer.point_light 1 1 1
w[2]← ⊂floor s1 s2

c← camera 400 300 (○÷3)
⍝ c[camera_transform]← ⊂view_transform (point 0.1 0 ¯10) (point 0 0 0) (vector 0 1 0)
⍝c[camera_transform]← ⊂view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)
⍝c[camera_transform]← ⊂view_transform (point 0.1 1.5 0.25) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_transform]← ⊂view_transform (point 0.1 1.5 ¯0.25) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂⌹view_transform (point 0.1 1.5 ¯0.25) (point 0 1 0) (vector 0 1 0)

#

In [30]:
im← w render #.time c

01:38.50

In [31]:
ppm← canvas_to_ppm im
'spheres.ppm' savePPM ppm

1512189

Here's the resulting scene!
<img src="spheres.jpg">

Here's the scene from the book, taken from the web site

http://forum.raytracerchallenge.com/thread/4/reflection-refraction-scene-description

```
# ======================================================
# reflect-refract.yml
#
# This file describes the scene illustrated at the start
# of chapter 11, "Reflection and Refraction", in "The
# Ray Tracer Challenge"
#
# by Jamis Buck <jamis@jamisbuck.org>
# ======================================================

# ======================================================
# the camera
# ======================================================

- add: camera
  width: 400
  height: 200
  field-of-view: 1.152
  from: [-2.6, 1.5, -3.9]
  to: [-0.6, 1, -0.8]
  up: [0, 1, 0]

# ======================================================
# light sources
# ======================================================

- add: light
  at: [-4.9, 4.9, -1]
  intensity: [1, 1, 1]

# ======================================================
# define constants to avoid duplication
# ======================================================

- define: wall-material
  value:
    pattern:
      type: stripes
      colors:
        - [0.45, 0.45, 0.45]
        - [0.55, 0.55, 0.55]
      transform:
        - [ scale, 0.25, 0.25, 0.25 ]
        - [ rotate-y, 1.5708 ]
    ambient: 0
    diffuse: 0.4
    specular: 0
    reflective: 0.3

# ======================================================
# describe the elements of the scene
# ======================================================

# the checkered floor
- add: plane
  transform:
    - [ rotate-y, 0.31415 ]
  material:
    pattern:
      type: checkers
      colors:
        - [0.35, 0.35, 0.35]
        - [0.65, 0.65, 0.65]
    specular: 0
    reflective: 0.4

# the ceiling
- add: plane
  transform:
    - [ translate, 0, 5, 0 ]
  material:
    color: [0.8, 0.8, 0.8]
    ambient: 0.3
    specular: 0

# west wall
- add: plane
  transform:
    - [ rotate-y, 1.5708 ] # orient texture
    - [ rotate-z, 1.5708 ] # rotate to vertical
    - [ translate, -5, 0, 0 ]
  material: wall-material

# east wall
- add: plane
  transform:
    - [ rotate-y, 1.5708 ] # orient texture
    - [ rotate-z, 1.5708 ] # rotate to vertical
    - [ translate, 5, 0, 0 ]
  material: wall-material

# north wall
- add: plane
  transform:
    - [ rotate-x, 1.5708 ] # rotate to vertical
    - [ translate, 0, 0, 5 ]
  material: wall-material

# south wall
- add: plane
  transform:
    - [ rotate-x, 1.5708 ] # rotate to vertical
    - [ translate, 0, 0, -5 ]
  material: wall-material

# ----------------------
# background balls
# ----------------------

- add: sphere
  transform:
    - [ scale, 0.4, 0.4, 0.4 ]
    - [ translate, 4.6, 0.4, 1 ]
  material:
    color: [0.8, 0.5, 0.3]
    shininess: 50

- add: sphere
  transform:
    - [ scale, 0.3, 0.3, 0.3 ]
    - [ translate, 4.7, 0.3, 0.4 ]
  material:
    color: [0.9, 0.4, 0.5]
    shininess: 50

- add: sphere
  transform:
    - [ scale, 0.5, 0.5, 0.5 ]
    - [ translate, -1, 0.5, 4.5 ]
  material:
    color: [0.4, 0.9, 0.6]
    shininess: 50

- add: sphere
  transform:
    - [ scale, 0.3, 0.3, 0.3 ]
    - [ translate, -1.7, 0.3, 4.7 ]
  material:
    color: [0.4, 0.6, 0.9]
    shininess: 50

# ----------------------
# foreground balls
# ----------------------

# red sphere
- add: sphere
  transform:
    - [ translate, -0.6, 1, 0.6 ]
  material:
    color: [1, 0.3, 0.2]
    specular: 0.4
    shininess: 5

# blue glass sphere
- add: sphere
  transform:
    - [ scale, 0.7, 0.7, 0.7 ]
    - [ translate, 0.6, 0.7, -0.6 ]
  material:
    color: [0, 0, 0.2]
    ambient: 0
    diffuse: 0.4
    specular: 0.9
    shininess: 300
    reflective: 0.9
    transparency: 0.9
    refractive-index: 1.5

# green glass sphere
- add: sphere
  transform:
    - [ scale, 0.5, 0.5, 0.5 ]
    - [ translate, -0.7, 0.5, -0.8 ]
  material:
    color: [0, 0.2, 0]
    ambient: 0
    diffuse: 0.4
    specular: 0.9
    shininess: 300
    reflective: 0.9
    transparency: 0.9
    refractive-index: 1.5
    ```

In [32]:
)CS #
⎕PATH←'RayTracer'

⍝ Wall material
wm← material
wm[RayTracer.material_ambient RayTracer.material_diffuse]← 0 0.4
wm[RayTracer.material_specular RayTracer.material_reflective]← 0 0.3
p← 0.45 0.45 0.45 stripe_pattern 0.55 0.55 0.55
p← p add_pattern_transform (scaling 0.25 0.25 0.25) +.× rotation_y 1.5708
wm[RayTracer.material_pattern]← ⊂p

⍝ Checkered floor
floor← plane
floor← floor add_shape_transform rotation_y 0.31415
(RayTracer.obj_material RayTracer.material_pattern⊃floor)← 0.35 0.35 0.35 checker_pattern 0.65 0.65 0.65
(RayTracer.obj_material RayTracer.material_specular⊃floor)← 0
(RayTracer.obj_material RayTracer.material_reflective⊃floor)← 0.4
⍝ Ceiling
ceiling← plane
ceiling← ceiling add_shape_transform translation 0 5 0
(RayTracer.obj_material RayTracer.material_color⊃ceiling)← 0.8 0.8 0.8
(RayTracer.obj_material RayTracer.material_ambient⊃ceiling)← 0.3
(RayTracer.obj_material RayTracer.material_specular⊃ceiling)← 0
idx←  ⊂(RayTracer.obj_material RayTracer.material_color)
idx,← ⊂(RayTracer.obj_material RayTracer.material_ambient)
idx,← ⊂(RayTracer.obj_material RayTracer.material_specular)
ceiling[idx]← (0.8 0.8 0.8) 0.3 0
⍝ west wall
west← plane
west← west add_shape_transform (translation ¯5 0 0)+.×(rotation_z 1.5708)+.×rotation_y 1.5708
west[RayTracer.obj_material]← ⊂wm
⍝ east wall
east← plane
east← east add_shape_transform (translation 5 0 0)+.×(rotation_z 1.5708)+.×rotation_y 1.5708
east[RayTracer.obj_material]← ⊂wm
⍝ north wall
north← plane
north← north add_shape_transform (translation 0 0 5)+.×rotation_x 1.5708
north[RayTracer.obj_material]← ⊂wm
⍝ south wall
south← plane
south← south add_shape_transform (translation 0 0 ¯5)+.×rotation_x 1.5708
south[RayTracer.obj_material]← ⊂wm

⍝ background balls
ball1← sphere
ball1← ball1 add_shape_transform (translation 4.6 0.4 1) +.× scaling 0.4 0.4 0.4
(RayTracer.obj_material RayTracer.material_color⊃ball1)←0.8 0.5 0.3
(RayTracer.obj_material RayTracer.material_shininess⊃ball1)←50

ball2← sphere
ball2← ball2 add_shape_transform (translation 4.7 0.3 0.4) +.× scaling 0.3 0.3 0.3
(RayTracer.obj_material RayTracer.material_color⊃ball2)←0.9 0.4 0.5
(RayTracer.obj_material RayTracer.material_shininess⊃ball2)←50

ball3← sphere
ball3← ball3 add_shape_transform (translation ¯1 0.5 4.5) +.× scaling 0.5 0.5 0.5
(RayTracer.obj_material RayTracer.material_color⊃ball3)←0.4 0.9 0.6
(RayTracer.obj_material RayTracer.material_shininess⊃ball3)←50

ball4← sphere
ball4← ball4 add_shape_transform (translation ¯1.7 0.3 4.7) +.× scaling 0.3 0.3 0.3
(RayTracer.obj_material RayTracer.material_color⊃ball4)←0.4 0.6 0.9
(RayTracer.obj_material RayTracer.material_shininess⊃ball4)←50

⍝ Foreground balls
redb← sphere
redb← redb add_shape_transform translation ¯0.6 1 0.6
(RayTracer.obj_material RayTracer.material_color⊃redb)← 1 0.3 0.2
(RayTracer.obj_material RayTracer.material_specular⊃redb)← 0.4
(RayTracer.obj_material RayTracer.material_shininess⊃redb)← 5

blueb← sphere
blueb← blueb add_shape_transform (translation 0.6 0.7 ¯0.6) +.× scaling 0.7 0.7 0.7
m← material
m[RayTracer.material_color]← ⊂0 0 0.2
m[RayTracer.material_ambient RayTracer.material_diffuse RayTracer.material_specular]← 0 0.4 0.9
m[RayTracer.material_shininess RayTracer.material_reflective]← 300 0.9
m[RayTracer.material_transparency RayTracer.material_refractive]← 0.9 1.5
⍝m[material_shininess material_reflective material_transparency material_refractive]← 300 0 0.9 1.5
blueb[RayTracer.obj_material]←⊂m

greenb← sphere
greenb← greenb add_shape_transform (translation ¯0.7 0.5 ¯0.8) +.× scaling 0.5 0.5 0.5
m← material
m[RayTracer.material_color]← ⊂0 0.2 0
m[RayTracer.material_ambient RayTracer.material_diffuse RayTracer.material_specular]← 0 0.4 0.9
m[RayTracer.material_shininess RayTracer.material_reflective]← 300 0.9
m[RayTracer.material_transparency RayTracer.material_refractive]← 0.9 1.5
⍝m[material_shininess material_reflective material_transparency material_refractive]← 300 0 0.9 1.5
greenb[RayTracer.obj_material]←⊂m


#

In [33]:
w← world
w[1]← ⊂⊂ (point ¯4.9 4.9 ¯1) RayTracer.point_light 1 1 1
w[2]← ⊂  floor west east north south ball1 ball2 ball3 ball4 redb blueb greenb

c← camera 800 600 1.152
c[RayTracer.camera_transform]← ⊂ view_transform (point ¯2.6 1.5 ¯3.9) (point ¯0.6 1 ¯0.8) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point ¯2.6 1.5 ¯3.9) (point ¯0.6 1 ¯0.8) (vector 0 1 0)


In [34]:
im← w render #.time c

17:50.49

In [35]:
ppm← canvas_to_ppm im
'Chap11.ppm' savePPM ppm

6048189

Shazzam! Looks really good!

<img src="Chap11.jpg">

# Chapter 12 Demo
Here's a scene using the new shape Cube.

In [36]:
)CS #
⎕PATH← '#.RayTracer'
floor← plane
m← material
⍝ m[material_color]←  ⊂1 0.9 0.9
p← 0 1 0.7 gradient_pattern 1 0.2 0.3
idx←  ⊂(RayTracer.obj_material RayTracer.material_color)
idx,← ⊂(RayTracer.obj_material RayTracer.material_specular)
idx,← ⊂(RayTracer.obj_material RayTracer.material_reflective)
floor[idx]← (p) 0 0.9

middle← cube
⍝ middle[obj_transform]← ⊂(rotation_y ○ 0.25) +.× translation ¯0.5 1 1
middle← middle add_shape_transform (rotation_y ○ 0.25) +.× translation ¯0.5 1 1
m← material
⍝ m[material_color]← ⊂0.1 1 0.5
p← 1 0 1 checker_pattern 0 1 0
⍝ p[pattern_transform]← ⊂scaling 0.1 0.1 0.1
p← p add_pattern_transform scaling 0.1 0.1 0.1
⍝ m[material_color]← ⊂p
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ middle[obj_material]← ⊂m
idx←  ⊂(RayTracer.obj_material RayTracer.material_color)
idx,← ⊂(RayTracer.obj_material RayTracer.material_diffuse)
idx,← ⊂(RayTracer.obj_material RayTracer.material_specular)
middle[idx]← (p) 0.7 0.3

right← cube
⍝ right[obj_transform]← ⊂ (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
right← right add_shape_transform (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
m← material
⍝ m[material_color]← ⊂0.5 1 0.1
⍝ p← 0 1 0 ring_pattern 0 0 0
⍝ p[pattern_transform]← ⊂(rotation_x ○ 0.5) +.× scaling 0.1 0.1 0.1
⍝ m[material_color]← ⊂p
m[RayTracer.material_color]← ⊂1 1 1 ⍝⊂0.3 0 0
m[RayTracer.material_ambient RayTracer.material_diffuse RayTracer.material_specular]← 0.1 0.9 0.9
m[RayTracer.material_shininess RayTracer.material_reflective]← 200 0.9
m[RayTracer.material_transparency RayTracer.material_refractive]← 1 1.5
right[RayTracer.obj_material]← ⊂m

left← cube
⍝ left[obj_transform]← ⊂ (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
left← left add_shape_transform (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
m← material
⍝ m[material_color]← ⊂1 0.8 0.1
p← 0 1 0 ring_pattern 0 0 0
⍝ p[pattern_transform]← ⊂(rotation_x ○ 0.5) +.× scaling 0.1 0.1 0.1
p← p add_pattern_transform (rotation_x ○ 0.5) +.× scaling 0.1 0.1 0.1
m[RayTracer.material_color]← ⊂p
m[RayTracer.material_diffuse]← 0.7
m[RayTracer.material_specular]← 0.3
left[RayTracer.obj_material]← ⊂m

w← world
w[1]← ⊂⊂ (point ¯10 10 ¯10) RayTracer.point_light 1 1 1
w[2]← ⊂  floor middle left right

c← camera 800 600 (○ ÷3)
c[RayTracer.camera_transform]← ⊂ view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)


#

In [37]:
⍝ im← w render c
im← w render #.time c

03:23.54

In [38]:
ppm← canvas_to_ppm im
'Chap12.ppm' savePPM ppm

6048189

Nifty!

<img src="Chap12.gif">

# Chapter 13 Demo
Adding cylinders and cones.

In [39]:
)CS #
⎕PATH← '#.RayTracer'
TRUE← 1
floor← plane
m← material
m[RayTracer.material_color]←  ⊂1 0.9 0.9
⍝ m[material_color]←  ⊂0 0 0 gradient_pattern 1 1 1 ⍝ ⊂1 0.9 0.9
m[RayTracer.material_specular]← 0
m[RayTracer.material_reflective]← 0.9
floor[RayTracer.obj_material]← ⊂m

middle← cylinder
middle[RayTracer.cylinder_minimum RayTracer.cylinder_maximum RayTracer.cylinder_closed]← 0 1 TRUE
⍝ middle[obj_transform]← ⊂translation ¯0.5 1 0.5
middle← middle add_shape_transform translation ¯0.5 1 0.5
m← material
⍝ m[material_color]← ⊂0.1 1 0.5
p← 1 0 1 checker_pattern 0 1 0
⍝ p[pattern_transform]← ⊂scaling 0.1 0.1 0.1
p← p add_pattern_transform scaling 0.1 0.1 0.1
m[RayTracer.material_color]← ⊂p
m[RayTracer.material_diffuse]← 0.7
m[RayTracer.material_specular]← 0.3
middle[RayTracer.obj_material]← ⊂m

right← cylinder
right[RayTracer.cylinder_minimum RayTracer.cylinder_maximum RayTracer.cylinder_closed]← 1 2 TRUE
⍝ right[obj_transform]← ⊂ (rotation_x ○ 0.5) +.× (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
right← right add_shape_transform (rotation_x ○ 0.5) +.× (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
m← material
⍝ m[material_color]← ⊂0.5 1 0.1
p← 0 1 0 ring_pattern 0 0 0
⍝ p[pattern_transform]← ⊂(rotation_x ○ 0.25) +.× scaling 0.1 0.1 0.1
p← p add_pattern_transform (rotation_x ○ 0.25) +.× scaling 0.1 0.1 0.1
m[RayTracer.material_color]← ⊂p
m[RayTracer.material_diffuse RayTracer.material_specular RayTracer.material_reflective]← 0.7 0.3 0.5
right[RayTracer.obj_material]← ⊂m

left← cone
left[RayTracer.cone_minimum RayTracer.cone_maximum RayTracer.cone_closed]← ¯1 1 TRUE
⍝ left[obj_transform]← ⊂ (translation ¯1.5 0.75 ¯0.75) +.× scaling 0.33 0.33 0.33
left← left add_shape_transform (translation ¯1.5 0.75 ¯0.75) +.× scaling 0.33 0.33 0.33
m← material
m[RayTracer.material_color RayTracer.material_diffuse RayTracer.material_specular]← (1 0.8 0.1) 0.7 0.3
left[RayTracer.obj_material]← ⊂m

w← world
w[1]← ⊂⊂ (point ¯10 10 ¯10) RayTracer.point_light 1 1 1
w[2]← ⊂  floor middle left right

c← camera 400 200 (○ ÷3)
c[RayTracer.camera_transform]← ⊂ view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)


#

In [40]:
im← w render #.time c

21.39

In [41]:
ppm← canvas_to_ppm im
'Chap13.ppm' savePPM ppm

1008189

And the result looks like

<img src="Chap13.gif">

Here's the demo code from the authors web site

http://forum.raytracerchallenge.com/thread/7/cylinders-scene-description

```
# ======================================================
# cylinders.yml
#
# This file describes the scene illustrated at the start
# of chapter 13, "Cylinders", in "The Ray Tracer
# Challenge"
#
# by Jamis Buck <jamis@jamisbuck.org>
# ======================================================

# ======================================================
# the camera
# ======================================================

- add: camera
  width: 400
  height: 200
  field-of-view: 0.314
  from: [8, 3.5, -9]
  to: [0, 0.3, 0]
  up: [0, 1, 0]

# ======================================================
# light sources
# ======================================================

- add: light
  at: [1, 6.9, -4.9]
  intensity: [1, 1, 1]

# ======================================================
# describe the elements of the scene
# ======================================================

# floor
- add: plane
  material:
    pattern:
      type: checkers
      colors:
        - [0.5, 0.5, 0.5]
        - [0.75, 0.75, 0.75]
      transform:
        - [ scale, 0.25, 0.25, 0.25 ]
        - [ rotate-y, 0.3 ]
    ambient: 0.2
    diffuse: 0.9
    specular: 0

- add: cylinder
  min: 0
  max: 0.75
  closed: true
  transform:
    - [ scale, 0.5, 1, 0.5 ]
    - [ translate, -1, 0, 1 ]
  material:
    color: [0, 0, 0.6]
    diffuse: 0.1
    specular: 0.9
    shininess: 300
    reflective: 0.9

# concentric cylinders

- add: cylinder
  min: 0
  max: 0.2
  closed: false
  transform:
    - [ scale, 0.8, 1, 0.8 ]
    - [ translate, 1, 0, 0 ]
  material:
    color: [1, 1, 0.3]
    ambient: 0.1
    diffuse: 0.8
    specular: 0.9
    shininess: 300

- add: cylinder
  min: 0
  max: 0.3
  closed: false
  transform:
    - [ scale, 0.6, 1, 0.6 ]
    - [ translate, 1, 0, 0 ]
  material:
    color: [1, 0.9, 0.4]
    ambient: 0.1
    diffuse: 0.8
    specular: 0.9
    shininess: 300

- add: cylinder
  min: 0
  max: 0.4
  closed: false
  transform:
    - [ scale, 0.4, 1, 0.4 ]
    - [ translate, 1, 0, 0 ]
  material:
    color: [1, 0.8, 0.5]
    ambient: 0.1
    diffuse: 0.8
    specular: 0.9
    shininess: 300

- add: cylinder
  min: 0
  max: 0.5
  closed: true
  transform:
    - [ scale, 0.2, 1, 0.2 ]
    - [ translate, 1, 0, 0 ]
  material:
    color: [1, 0.7, 0.6]
    ambient: 0.1
    diffuse: 0.8
    specular: 0.9
    shininess: 300

# decorative cylinders
- add: cylinder
  min: 0
  max: 0.3
  closed: true
  transform:
    - [ scale, 0.05, 1, 0.05 ]
    - [ translate, 0, 0, -0.75 ]
  material:
    color: [1, 0, 0]
    ambient: 0.1
    diffuse: 0.9
    specular: 0.9
    shininess: 300

- add: cylinder
  min: 0
  max: 0.3
  closed: true
  transform:
    - [ scale, 0.05, 1, 0.05 ]
    - [ translate, 0, 0, 1.5 ]
    - [ rotate-y, -0.15 ]
    - [ translate, 0, 0, -2.25 ]
  material:
    color: [1, 1, 0]
    ambient: 0.1
    diffuse: 0.9
    specular: 0.9
    shininess: 300

- add: cylinder
  min: 0
  max: 0.3
  closed: true
  transform:
    - [ scale, 0.05, 1, 0.05 ]
    - [ translate, 0, 0, 1.5 ]
    - [ rotate-y, -0.3 ]
    - [ translate, 0, 0, -2.25 ]
  material:
    color: [0, 1, 0]
    ambient: 0.1
    diffuse: 0.9
    specular: 0.9
    shininess: 300

- add: cylinder
  min: 0
  max: 0.3
  closed: true
  transform:
    - [ scale, 0.05, 1, 0.05 ]
    - [ translate, 0, 0, 1.5 ]
    - [ rotate-y, -0.45 ]
    - [ translate, 0, 0, -2.25 ]
  material:
    color: [0, 1, 1]
    ambient: 0.1
    diffuse: 0.9
    specular: 0.9
    shininess: 300

# glass cylinder
- add: cylinder
  min: 0.0001
  max: 0.5
  closed: true
  transform:
    - [ scale, 0.33, 1, 0.33 ]
    - [ translate, 0, 0, -1.5 ]
  material:
    color: [0.25, 0, 0]
    diffuse: 0.1
    specular: 0.9
    shininess: 300
    reflective: 0.9
    transparency: 0.9
    refractive-index: 1.5
```

In [42]:
)CS #
⎕PATH← '#.RayTracer'


#

# Chapter 14 Demo
Here's a simple example of grouping shapes together and applying a group transform to all. I've implemented groups differently that what the book author proposed. The key is to apply the **flatten_group** function to groups before adding them to a world scene. This way the group transform is precomputed rather than computed with each ray being cast.

In [43]:
)CS #
⎕PATH← '#.RayTracer'

floor← plane
m← material
m[RayTracer.material_color RayTracer.material_specular RayTracer.material_reflective]←  (1 0.9 0.9) 0 0.9
⍝ m[material_specular]← 0
⍝ m[material_reflective]← 0.9
floor[RayTracer.obj_material]← ⊂m

middle← cube
⍝ middle[obj_transform]← ⊂translation ¯0.5 1 1
middle← middle add_shape_transform translation ¯0.5 1 1
m← material
m[RayTracer.material_color RayTracer.material_diffuse RayTracer.material_specular]← (0.1 1 0.5) 0.7 0.3
⍝ p← 1 0 1 checker_pattern 0 1 0
⍝ p[pattern_transform]← ⊂scaling 0.1 0.1 0.1
⍝ m[material_color]← ⊂p
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
middle[RayTracer.obj_material]← ⊂m

right← cube
⍝ right[obj_transform]← ⊂ (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
right← right add_shape_transform (translation 1.5 0.5 ¯0.5) +.× scaling 0.5 0.5 0.5
m← material
⍝ p← 0 1 0 ring_pattern 0 0 0
⍝ p[pattern_transform]← ⊂(rotation_x ○ 0.5) +.× scaling 0.1 0.1 0.1
⍝ m[material_color]← ⊂p
m[RayTracer.material_color RayTracer.material_diffuse]← (0.5 1 0.1) 0.7
m[RayTracer.material_specular RayTracer.material_reflective]← 0.3 0.5
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
⍝ m[material_reflective]← 0.5
right[RayTracer.obj_material]← ⊂m

left← cube
⍝ left[obj_transform]← ⊂ (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
left← left add_shape_transform (translation ¯1.5 0.33 ¯0.75) +.× scaling 0.33 0.33 0.33
m← material
m[RayTracer.material_color RayTracer.material_diffuse RayTracer.material_specular]← (1 0.8 0.1) 0.7 0.3
⍝ m[material_diffuse]← 0.7
⍝ m[material_specular]← 0.3
left[RayTracer.obj_material]← ⊂m

g← group
⍝ g[obj_transform]← ⊂rotation_y ○ 0.25
g← g add_shape_transform rotation_y ○ 0.25
g[RayTracer.group_members],← ⊂⊂left
g[RayTracer.group_members],← ⊂⊂middle
g[RayTracer.group_members],← ⊂⊂right

w← world
w[1]← ⊂⊂ (point ¯10 10 ¯10) RayTracer.point_light 1 1 1
w[2],← ⊂⊂  floor
w[2],← ⊂ g flatten_group identity4
⍝w

c← camera 400 200 (○ ÷3)
c[RayTracer.camera_transform]← ⊂ view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)
c[RayTracer.camera_inverse]← ⊂ ⌹view_transform (point 0 1.5 ¯5) (point 0 1 0) (vector 0 1 0)


#

In [44]:
im← w render #.time c

19.19

In [45]:
ppm← canvas_to_ppm im
'Chap14.ppm' savePPM ppm

1008189

Here's the result!
<img src="Chap14.gif">